## Algorithm Comparison for BPM Analysis
by Marc Jones, written for python 3.6+

This notebook undertakes a comparative analysis for three of Essentia's different of BPM estimation algorithms:
* Multi-Feature Beat Tracking (Zapata, Davies, & Gomez 2014)
* Reliability-informed Beat Tracking (Degara, Rua, Pena, Torres-Guiharro, Davies, & Plumbley 2012)
* Streamlined Tempo Estimation Based on Autocorrelation and Cross-correlation With Pulses (Percival & Tzanetakis 2014)

Evaluation of the algorithms are in terms of runtime and performance accuracy. In order to evaluate the various algorithms we can use a large annoted [dataset](https://github.com/GiantSteps/giantsteps-tempo-dataset) (including genre, bpm, and audio) created for the collaborative '[GiantSteps](https://www.upf.edu/web/mtg/giant-steps)' project. Having the ground truths of the provided audiofiles allows us to test the accuracy of each algorithm.

#### Multi-Feature Beat Tracking System
The Multi-Feature Beat Tracking sysyem is composed of three parts, first a set of onset detection functions, followed by beat period estimation and beat tracking for each onset detection function used. In total nine onset detection functions were used: __energy flux, spectral flux, spectral flux lof filtered, complex spectral difference, beat emphasis function, harmonic feature, mel auditory feature, phase slope function, & bandwise accent signals__. Lastly in a committee like approach - bpm is chosen using a selection from the estimated beat locations using the various used onset detection functions

![alt-text](https://i.imgur.com/V0UeuLD.png)
_Diagram from Zapata, Davies, & Gomez 2014_

#### Degara Beat Tracking
Rather than tracking beat time instants, the Degara beat tracking system uses __beat phase__ and __beat period__ salience as feature inputs from which a __beat period__ is calculated. Then, a beat tracking probabilistic model takes the phase observation and the beat period estimation to return a set of beat time estimates. Lastly, the beat period salience feature is assed using k-NN to measure the reliability of the aforementioned beat period estimations.

![alt-text](https://i.imgur.com/KRsuhn8.png)
_Diagram from Degara, Rua, Pena, Torres-Guiharro, Davies, & Plumbley 2012_

#### Percival Beat Tracking
The Percival Beat Tracking algorithm uses a time-domain __'Onset Strength Signal'__ from the input audio to identify rhythmically salient events. They consider beats to be generally located near onsets and view the OSS as a semi regular subset of the onsets. The onsets provided are then passed through an auto-correlation function where the top-10 peaks are selected and evaluated against a __pulse train__, which is the corerlation of the OSS with the 10 estimated tempos. The top scoring estimated tempo is then returned.

![alt-text](https://i.imgur.com/deV6JG0.png)
_Diagram from Percival & Tzanetakis 2014_

In [1]:
# Import necessary libraries and packages
import os, sys, time, copy as cpy
from essentia.standard import *
from essentia import Pool, array
import numpy as np
from bokeh.plotting import output_notebook,figure,show
from bokeh.layouts import column
import warnings
output_notebook()
warnings.filterwarnings(action='ignore')

Loading BokehJS ...

### File Organization
The aformentioned [dataset](https://github.com/GiantSteps/giantsteps-tempo-dataset) needs to be downloaded in pieces. While the annotated ground truth values are included in the provided link, the actual audio files (~1GB worth of mp3s) needed for testing have to be scraped from Beatport using the included shell script __'audio_dl.sh'__ (run from terminal by typing 'bash audio_dl.sh'). The resulting __'giantsteps-tempo-dataset'__ directory should include the __'annotations'__ and '__audio__' folders in order for this notebook to work successfully.

In [2]:
# Organize file paths for audio files and corresponding ground truth BPMs
mp3_paths = []
gt_bpm_values = []
data_dir = 'giantsteps-tempo-dataset'

tempo_dir = data_dir+'/annotations/tempo'
for gt in os.listdir(tempo_dir):
    if not gt.startswith('.'):
        gt_bpm_values.append(open(tempo_dir+'/'+gt, 'r').read())
        
audio_dir = data_dir+'/audio'
for mp3 in os.listdir(audio_dir):
    if not mp3.startswith('.'):
        mp3_paths.append(audio_dir+'/'+mp3)

### BPM Extraction

Included with this notbook should be a file called __'extracted_bpm_data.txt'__, it contains the extracted BPM values for each audio file using each of the aforementioned extraction algorithms. If you don't have that file this program will create it for you iterating over each of the 664 audio files in the dataset, using Essentia to gather the data, then outputting that information to file. This process is not quick and is recommended that you run the cell below overnight as performance may vary greatly from machine to machine. 

In [3]:
# Init dictionary to store lists of bpm values & errors by algorithm
bpm_by_algo, error_by_algo, uncorrected_error_by_algo = ({'MultiFeature':[],'Degara':[],'Percival':[]} for i in range(3))

# First check to see if file with extracted data file has already been created
if os.path.isfile('extracted_bpm_data.txt'):
#     gt_bpm_values = []
    file = open('extracted_bpm_data.txt','r')
    for i,line in enumerate(file):
#         print(line)
        line = line.replace('[','').replace(']','').replace(' ','').replace('\n','').replace('\'','').split(',')
        gt_bpm = gt_bpm_values[i]
        mf_bpm = line[0] ; dg_bpm = line[1] ; pe_bpm = line[2]
        print('#'+str(i+1)+' Ground Truth: '+str(gt_bpm)+'\t MultiFeature: '+ mf_bpm + '\t Degara: '+ dg_bpm + ' \t Percival: '+ pe_bpm)
        bpm_by_algo['MultiFeature'].append(float(mf_bpm))
        bpm_by_algo['Degara'].append(float(dg_bpm))
        bpm_by_algo['Percival'].append(float(pe_bpm))
#Otherwise begin the extraction process
else:
    output_file = open('extracted_bpm_data.txt','w')
    # Extract bpm using a variety of algorithms 
    for i, audiofile in enumerate(mp3_paths):
        audio = MonoLoader(filename = audiofile)()    

        # RhythmExtractor2013 uses Beat Tracking, with either 'multifeature' (Zapata 2014) or 'degara' (Degara 2012) methods
        rhythm_extractor = RhythmExtractor2013( method = 'multifeature' )
        degara_rhythm_extractor = RhythmExtractor2013( method = 'degara' )
        mf_bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)
        dg_bpm, dbeats, dbeats_confidence, d_, dbeats_intervals = degara_rhythm_extractor(audio)
        mf_bpm = round(mf_bpm,3)
        dg_bpm = round(dg_bpm,3)

        # Percival BPM Estimator (Percival & Tzanetakis 2014)
        percival_bpm_estimator = PercivalBpmEstimator()
        pe_bpm = percival_bpm_estimator(audio)
        pe_bpm = round(pe_bpm,3)    

        # add values to dict
        bpm_by_algo['MultiFeature'].append(float(mf_bpm))
        bpm_by_algo['Degara'].append(float(dg_bpm))
        bpm_by_algo['Percival'].append(float(pe_bpm))

        # output results to console and to file
        output = [mf_bpm,dg_bpm,pe_bpm]
        output_file.write(str(output)+'\n')
        print('Track #'+str(i+1)+' Ground Truth: '+ str(gt_bpm_values[i]) + '\t MultiFeature: '+ str(mf_bpm) + '\t Degara: '+ str(dg_bpm) + ' \t Percival: '+ str(pe_bpm))
    output_file.close()

uncorrected_bpm_by_algo  = cpy.deepcopy(bpm_by_algo)

#1 Ground Truth: 126	 MultiFeature: 126.002	 Degara: 125.984 	 Percival: 84.032
#2 Ground Truth: 170	 MultiFeature: 172.266	 Degara: 172.266 	 Percival: 87.593
#3 Ground Truth: 140	 MultiFeature: 140.009	 Degara: 140.01 	 Percival: 139.675
#4 Ground Truth: 136	 MultiFeature: 135.808	 Degara: 135.808 	 Percival: 135.999
#5 Ground Truth: 86	 MultiFeature: 86.106	 Degara: 114.666 	 Percival: 86.133
#6 Ground Truth: 171.419	 MultiFeature: 172.266	 Degara: 114.317 	 Percival: 85.421
#7 Ground Truth: 174	 MultiFeature: 116.597	 Degara: 115.546 	 Percival: 86.857
#8 Ground Truth: 140	 MultiFeature: 140.032	 Degara: 139.933 	 Percival: 140.148
#9 Ground Truth: 172	 MultiFeature: 172.265	 Degara: 172.265 	 Percival: 86.133
#10 Ground Truth: 148	 MultiFeature: 147.598	 Degara: 147.598 	 Percival: 73.828
#11 Ground Truth: 140	 MultiFeature: 140.028	 Degara: 139.969 	 Percival: 69.837
#12 Ground Truth: 120	 MultiFeature: 119.988	 Degara: 119.988 	 Percival: 120.185
#13 Ground Truth: 140	 MultiFeat

NoveltyCurve & BpmHistogram BPM estimations performed markedly worse than the above three algorithms as result they have been excluded from the output but if you are interested in how to derive their estimations please look at the code below, it can be copied anywhere below the top line of the second 'for loop' in the above cell ( "audio = ..." ).

In [4]:
# NoveltyCurve & BpmHistogram BPM estimation code

#     frameSize = 2048
#     hopSize = 1024
#     weight = 'hybrid'
#     w = Windowing(type='hann')
#     s = Spectrum()
#     novelty_curve_bpm_estimator = NoveltyCurveFixedBpmEstimator()
#     frequency_bands = FrequencyBands()
    
#     bands_energies = []
#     for frame in FrameGenerator(audio, frameSize = frameSize, hopSize=hopSize):
#         bands_energies.append(frequency_bands(s(w(frame))))

#     novelty_curve = NoveltyCurve(frameRate = 44100/hopSize, weightCurveType = weight)(np.array(bands_energies))
    
#     bpm_from_novelty_curve, amplitudes = novelty_curve_bpm_estimator(novelty_curve)    
#     bpm_from_bpm_histogram, candidates, magnitudes, tempogram, _, ticks, ticks_strength, sinusoid = BpmHistogram(frameRate=44100/hopSize)(novelty_curve)
    
#     print(bpm_from_bpm_histogram)
#     print(bpm_from_novelty_curve)    

### Correcting Estimations That Might Be Accurate

In determining the accuracy/error levels of the BPM estimation there requires a bit of nuance to identifying because the algorithms we use to extract that information are not necessarily precise; for example: a track with a ground truth BPM at 160 might be analyzed and determined to have a BPM of 80, and if the only perceptable onsets from which these algorithms derive thier information are at 1/2 the tempo of the ground truth, then an estimation of 80 bpm should then be considered accurate and not an error. This problem with doubling/halving from the ground truth value is apparent in many of the estimations and needs to be taken into consideration.

I propose a means of classifying whether or not the errors are substantive:
- segregate the BPM values into two bins: 0-100 and 100-200
- identifying which of the two bins holds the values closer to the ground truth
- checking to see if there are halved/doubled BPM values in the opposite bin are actually accurate and closer to the ground truth (+/- 3bpm)
    - if yes, then the halved or doubled value is corrected by mutliplying or dividing it by two
    - if no, then the value is untouched as is

In [5]:
# plot bpm estimations before correction
x_axis = [i for i in range(664)]
fig_b = figure(title="Before Correction", plot_width=800,plot_height=300, x_axis_label='Song # (664 total)',y_axis_label='BPM')
fig_b.line(x_axis, gt_bpm_values, legend='Ground Truth', line_color='navy',line_width= 1.25)
fig_b.line(x_axis, uncorrected_bpm_by_algo['MultiFeature'], legend='Multi Feature', line_color='orange', line_alpha=.5)
fig_b.line(x_axis, uncorrected_bpm_by_algo['Degara'], legend='Degara', line_color='red', line_alpha=.5)
fig_b.line(x_axis, uncorrected_bpm_by_algo['Percival'], legend='Percival', line_color='purple', line_alpha=.5)
fig_b.circle(x_axis, gt_bpm_values, legend='Ground Truth', color='black', alpha= .5, size=1)
fig_b.circle(x_axis, uncorrected_bpm_by_algo['MultiFeature'], legend='Multi Feature', color='black', alpha=.5, size=1)
fig_b.circle(x_axis, uncorrected_bpm_by_algo['Degara'], legend='Degara', color='black', alpha=.5, size=1)
fig_b.circle(x_axis, uncorrected_bpm_by_algo['Percival'], legend='Percival', color='black', alpha=.5, size=1)

num_corrected = []
for i,gt in enumerate(gt_bpm_values):
    gt = float(gt)
    bin100 = []
    bin200 = []
    mf_bpm = bpm_by_algo['MultiFeature'][i]
    de_bpm = bpm_by_algo['Degara'][i]
    pe_bpm = bpm_by_algo['Percival'][i]
    
    # segregate values into two bins 0-100 and 100-200
    bin200.append(mf_bpm) if mf_bpm > 100 else bin100.append(mf_bpm)
    bin200.append(de_bpm) if de_bpm > 100 else bin100.append(de_bpm)
    bin200.append(pe_bpm) if pe_bpm > 100 else bin100.append(pe_bpm)

    # find values close to ground truth
    bin_vote = 0
    for val in bin100:
        if abs(val-gt) < 3:
            bin_vote -= 1
    for val in bin200:
        if abs(val-gt) < 3:
            bin_vote += 1
    
    # if abs(bin_vote) = 3 all estimations are accurate
    # if bin_vote = 2 the remaining value in bin100 needs to be doubled and checked for accuracy
    # if bin_vote = -2 the remaining value in bin200 needs to be halved and checked for accuracy
    # if bin_vote = 1 the remaining two values in bin100 needs to be doubled and checked for accuracy
    # if bin_vote = -1 the remaining two values in bin200 needs to be halved and checked for accuracy
    # if bin_vote = 0 : all estimations are inaccurate (more than +/- 3bpm away from truth)  
    
    # check doubled/halved values and correct them accordingly in the bpm_by_algo container
    if abs(bin_vote) == 2 or abs(bin_vote) == 1:
        if bin_vote > 0:
            for val in bin100:
                dbl_val = val*2
                if abs(dbl_val-gt) < 3:
                    num_corrected.append(i)
                    if mf_bpm == val:
                        bpm_by_algo['MultiFeature'][i] = dbl_val
                    if de_bpm == val:
                        bpm_by_algo['Degara'][i] = dbl_val
                    if pe_bpm == val:
                        bpm_by_algo['Percival'][i] = dbl_val
        else:
            for val in bin200:
                hlv_val = val/2
                if abs(hlv_val-gt) < 3:
                    num_corrected.append(i)
                    if mf_bpm == val:
                        bpm_by_algo['MultiFeature'][i] = hlv_val
                    if de_bpm == val:
                        bpm_by_algo['Degara'][i] = hlv_val
                    if pe_bpm == val:
                        bpm_by_algo['Percival'][i] = hlv_val
                        
print('Corrections made at indicies:',num_corrected)
# plot bpm estimations after correction
x_axis = [i for i in range(664)]
fig_a = figure(title="After Correction", plot_width=800,plot_height=300, x_axis_label='Song # (664 total)',y_axis_label='BPM')
# fig_a.circle(num_corrected, 0, size=2, color="black", legend='Correction')
fig_a.line(x_axis, gt_bpm_values, legend='Ground Truth', line_color='navy',line_width= 1.25)
fig_a.line(x_axis, bpm_by_algo['MultiFeature'], legend='Multi Feature', line_color='orange', line_alpha=.5)
fig_a.line(x_axis, bpm_by_algo['Degara'], legend='Degara', line_color='red', line_alpha=.5)
fig_a.line(x_axis, bpm_by_algo['Percival'], legend='Percival', line_color='purple', line_alpha=.5)
fig_a.circle(x_axis, gt_bpm_values, legend='Ground Truth', color='black', alpha= .5, size=1)
fig_a.circle(x_axis, bpm_by_algo['MultiFeature'], legend='Multi Feature', color='black', alpha=.5, size=1)
fig_a.circle(x_axis, bpm_by_algo['Degara'], legend='Degara', color='black', alpha=.5, size=1)
fig_a.circle(x_axis, bpm_by_algo['Percival'], legend='Percival', color='black', alpha=.5, size=1)
show(column(fig_b,fig_a))

Corrections made at indicies: [5, 8, 9, 10, 12, 16, 18, 26, 26, 34, 37, 39, 39, 45, 45, 46, 46, 58, 59, 66, 66, 69, 69, 78, 78, 80, 80, 83, 83, 85, 85, 88, 88, 90, 95, 97, 97, 107, 107, 110, 110, 111, 113, 115, 119, 129, 129, 133, 138, 138, 142, 142, 158, 158, 168, 172, 172, 175, 177, 178, 179, 182, 200, 209, 218, 218, 226, 226, 227, 230, 234, 234, 249, 249, 262, 267, 268, 268, 271, 276, 277, 277, 289, 289, 290, 292, 292, 297, 306, 307, 308, 317, 318, 324, 324, 333, 347, 352, 352, 362, 362, 378, 379, 379, 381, 385, 390, 390, 393, 398, 399, 409, 429, 429, 431, 431, 433, 433, 435, 451, 451, 458, 463, 474, 474, 481, 489, 489, 496, 497, 502, 507, 508, 508, 509, 509, 512, 543, 545, 563, 566, 569, 577, 580, 587, 587, 592, 592, 606, 606, 612, 612, 626, 631, 632, 641, 643, 650, 656, 660]


### Timing the Algorithms

The BPM extraction code earlier was written to reduce the amount of calls to Essentia's 'MonoLoader' class such that the our audio clip could stay in memory. Here we instead complete the tasks iteratively in order to time the extraction process for each algorithm - and ultimately compare them.

In [6]:
# Timing the algorithms
# multifeature
start_time = time.time()
mf_time = []
for i, audiofile in enumerate(mp3_paths):
    audio = MonoLoader(filename = audiofile)()    
    rhythm_extractor = RhythmExtractor2013( method = 'multifeature' )
    mf_bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)
    mf_time.append(time.time() - start_time)
    if i >= 20:
        break
mf_total_time = time.time() - start_time
print('--- MultiFeature: '+str(mf_total_time)+' seconds ---', )
        
# degara
start_time = time.time()
de_time = []
for i, audiofile in enumerate(mp3_paths):
    audio = MonoLoader(filename = audiofile)()    
    rhythm_extractor = RhythmExtractor2013( method = 'degara' )
    mf_bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)
    de_time.append(time.time() - start_time)
    if i >= 20:
        break
de_total_time = time.time() - start_time
print('--- Degara: '+str(de_total_time)+' seconds ---', )

# percival
start_time = time.time()
pe_time = []
for i, audiofile in enumerate(mp3_paths):
    audio = MonoLoader(filename = audiofile)()    
    percival_bpm_estimator = PercivalBpmEstimator()
    pe_bpm = percival_bpm_estimator(audio)
    pe_time.append(time.time() - start_time)
    if i >= 20:
        break
pe_total_time = time.time() - start_time
print('--- Percival: '+str(pe_total_time)+' seconds ---', )

# plot results
x_axis = [i for i in range(20)]
l = figure(title="Computation Time", plot_width=800,plot_height=300, x_axis_label='Song #',y_axis_label='Time')
l.line(x_axis, mf_time, legend='Multi Feature', line_color='orange', line_alpha=.75)
l.line(x_axis, de_time, legend='Degara', line_color='red', line_alpha=.75)
l.line(x_axis, pe_time, legend='Percival', line_color='purple', line_alpha=.75)
l.circle(x_axis, mf_time, legend='Multi Feature', color='black', alpha=.5, size=1)
l.circle(x_axis, de_time, legend='Degara', color='black', alpha=.5, size=1)
l.circle(x_axis, pe_time, legend='Percival', color='black', alpha=.5, size=1)
show(l)

--- MultiFeature: 78.23886585235596 seconds ---
--- Degara: 28.939376831054688 seconds ---
--- Percival: 39.39996862411499 seconds ---


### Error Calculations

While the Degara and Percival algorithms have computed the BPM estimates in approximately half (or less than) the amount of time it took the MultiFeature algorithm to compute BPM estimates, it remains to be seen if the speed tradeoff is worth the tradeoff in accuracy. 

In [7]:
#Here we take the absolute value between the difference of the ground truth and the extracted values
for i,gt_val in enumerate(gt_bpm_values):
    error_by_algo['MultiFeature'].append( abs( float(gt_val) - bpm_by_algo['MultiFeature'][i] ))
    error_by_algo['Degara'].append( abs( float(gt_val) - bpm_by_algo['Degara'][i] ))
    error_by_algo['Percival'].append( abs( float(gt_val) - bpm_by_algo['Percival'][i] ))
    uncorrected_error_by_algo['MultiFeature'].append( abs( float(gt_val) - uncorrected_bpm_by_algo['MultiFeature'][i] ))
    uncorrected_error_by_algo['Degara'].append( abs( float(gt_val) - uncorrected_bpm_by_algo['Degara'][i] ))
    uncorrected_error_by_algo['Percival'].append( abs( float(gt_val) - uncorrected_bpm_by_algo['Percival'][i] ))
    
mf_err_mean = round(np.mean(error_by_algo['MultiFeature']),3)
mf_err_std = round(np.std(error_by_algo['MultiFeature']),3)
dg_err_mean = round(np.mean(error_by_algo['Degara']),3)
dg_err_std = round(np.std(error_by_algo['Degara']),3)
pe_err_mean = round(np.mean(error_by_algo['Percival']),3)
pe_err_std = round(np.std(error_by_algo['Percival']),3)

mf_uc_err_mean = round(np.mean(uncorrected_error_by_algo['MultiFeature']),3)
mf_uc_err_std = round(np.std(uncorrected_error_by_algo['MultiFeature']),3)
dg_uc_err_mean = round(np.mean(uncorrected_error_by_algo['Degara']),3)
dg_uc_err_std = round(np.std(uncorrected_error_by_algo['Degara']),3)
pe_uc_err_mean = round(np.mean(uncorrected_error_by_algo['Percival']),3)
pe_uc_err_std = round(np.std(uncorrected_error_by_algo['Percival']),3)

print('Algorithm: \'MultiFeature\'')
print('\tcorrected error mean =',str(mf_err_mean),'\t\tbpm corrected error std  =',str(mf_err_std))
print('\tuncorrected error mean =',str(mf_uc_err_mean),'\t\tbpm uncorrected error std  =',str(mf_uc_err_std))
print('Algorithm:\'Degara\'')
print('\tcorrected error mean =',str(dg_err_mean),'\t\tbpm corrected error std  =',str(dg_err_std))
print('\tuncorrected error mean =',str(dg_uc_err_mean),'\t\tbpm uncorrected error std  =',str(dg_uc_err_std))
print('Algorithm: \'Percival\'')
print('\tcorrected error mean =',str(pe_err_mean),'\t\tbpm corrected error std  =',str(pe_err_std))
print('\tuncorrected error mean =',str(pe_uc_err_mean),'\t\tbpm uncorrected error std  =',str(pe_uc_err_std))

Algorithm: 'MultiFeature'
	corrected error mean = 23.268 		bpm corrected error std  = 32.408
	uncorrected error mean = 28.6 		bpm uncorrected error std  = 34.586
Algorithm:'Degara'
	corrected error mean = 23.175 		bpm corrected error std  = 31.537
	uncorrected error mean = 27.869 		bpm uncorrected error std  = 33.505
Algorithm: 'Percival'
	corrected error mean = 24.377 		bpm corrected error std  = 34.938
	uncorrected error mean = 32.781 		bpm uncorrected error std  = 37.388


### Conclusions and Future Work

The time needed to compute BPM estimates using the 'MultiFeature' algorithm not only far exceeded those of other two demonstrated here, it also did not outperform the other two algorithms once each BPM estimation was corrected for accurate mis-estimations. Ultimately the 'Degara' algorithm provided the best tradeoff between time and accuracy performance.

What might be worth looking into is to see how these algorithms perform differently based on genre, as genre is one of the annotations available with the dataset used for this project.